# 경로확인

In [23]:
import os

In [24]:
os.getcwd()

'C:\\Users\\administrator\\전처리데이터'

In [25]:
%cd C:\\Users\\administrator\\model_none_audio_ver0012

C:\Users\administrator\model_none_audio_ver0012


In [26]:
# data_path = 'C:\\Users\\administrator\\전처리데이터\\'

In [27]:
# model_path = 'C:\\Users\\administrator\\model_none_audio_ver0012\\'

# 1. 파일 읽기(구버전 파일 형식: .h5, 신버전 파일 형식 : .keras)

In [28]:
from keras.models import load_model

# 저장된 인코더 모델 불러오기
encoder_model = load_model('multi_encoder_model_200.h5')
decoder_model = load_model('multi_decoder_model_200.h5')

# 1.5 토크나이저 및 딕셔너리 로드

In [29]:
%cd C:\\Users\\administrator\\전처리데이터

C:\Users\administrator\전처리데이터


### 토크나이저

In [30]:
import tensorflow as tf
import json

# Decoder_모델의 Infernece 출력을 
# 저장된 토크나이저 파일을 읽습니다
with open('song_tokenizer_none_audio_30000.json', 'r', encoding='utf-8') as f:
    loaded_song_tokenizer_json = json.load(f)  # json.load를 사용하여 직접 JSON 객체로 변환

# JSON 객체에서 토크나이저를 복원합니다.
song_tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(loaded_song_tokenizer_json)

In [31]:
song_tokenizer.word_index['<sos>']

261226

In [32]:
song_tokenizer.word_index['<eos>']

261227

In [33]:
# playlist_title_tokenizer
from transformers import BertTokenizer

# Encoder_입력을 전처리(토큰화 및 전처리) 하기 위한 Bert 토크나이저
playlist_title_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### 딕셔너리

In [34]:
# JSON 파일로부터 식별자 딕셔너리 로드
# song(Decoder_dict)가 있어야만 반드시 출력 숫자-> '노래 제목'으로 출력할 수 있음! 필수
with open('song_identifiers_30000.json', 'r', encoding='utf-8') as file:
    song_identifiers = json.load(file)

len(song_identifiers)

261227

In [35]:
song_identifiers

{'Rainy Day': 'song_name_1',
 'When This Rain Stops': 'song_name_2',
 '밤편지': 'song_name_3',
 '우산 (Feat. 윤하)': 'song_name_4',
 '장마 (Rainy Season)': 'song_name_5',
 '비와 당신': 'song_name_6',
 '사랑이 이별이 돼 가는 모습이': 'song_name_7',
 '비가 오는 날엔': 'song_name_8',
 'Rainy day (Feat. ASH ISLAND, Skinny Brown)': 'song_name_9',
 '비오는 날 뭐해': 'song_name_10',
 'To Die For': 'song_name_11',
 'Sober': 'song_name_12',
 'Rain Drop': 'song_name_13',
 '비오는 날 듣고 싶은 노래': 'song_name_14',
 'Bambi': 'song_name_15',
 '우산 (Love Song)': 'song_name_16',
 '비가오면 비가오죠': 'song_name_17',
 'Umbrella (Feat. Ashley Alisha, prettyhappy)': 'song_name_18',
 'Paris in the Rain': 'song_name_19',
 '비가 오는 날엔 (2021)': 'song_name_20',
 '장마 (Feat. 영준 of Brown Eyed Soul)': 'song_name_21',
 'Rain': 'song_name_22',
 '비 오는 날 듣기 좋은 노래 (Feat. Colde)': 'song_name_23',
 'Kiss The Rain': 'song_name_24',
 '비도 오고 그래서 (Feat. 신용재)': 'song_name_25',
 '비': 'song_name_26',
 '나의 바람 (Wind And Wish)': 'song_name_27',
 '슬픈 다짐': 'song_name_28',
 'One More Ti

In [36]:
song_identifiers['우리가 남긴 불빛']

'song_name_62520'

# 2. Inference 함수 선언 : 하이퍼 파라미터도 같이 설정

In [37]:
import numpy as np
# 하이퍼 파라미터 설정
## 입력 패딩 파라미터
max_playlist_title_length = 25
max_songs = 10

## 2차원 입력 데이터, feature_length
lyric_length = 150
#audio_length = 7

## 임베딩 레이어 파라미터
### 임베딩 출력
embed_output_dim = 128
embed_outpu_dim = 128
### 임베딩 어휘 갯수
### 인코더-디코더
playlist_vocab = 119548
song_vocab = 261228      # 노래제목의 갯수

### 임베딩에 속한 어휘(토큰) 갯수
artist_vocab = 64384
album_vocab = 167466
genre_vocab = 79
release_date_vocab = 58
lyric_vocab = 119517

## LSTM units 파라미터
lstm_units = 128

start_token_id = 261226
stop_token_id = 261227

def inference_songs(input_seq):
    # 인코더 모델을 사용하여 전체 출력과 상태 얻기
    encoder_output, state_h, state_c = encoder_model.predict(input_seq)
    states_value = [state_h, state_c]

    # 초기 입력 시퀀스 생성 (SOS 토큰)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = start_token_id

    # 예측을 위한 루프
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq, state_h, state_c, encoder_output])

        # 예측된 토큰 ID 가져오기
        sampled_token_id = np.argmax(output_tokens[0, -1, :])
        if sampled_token_id != stop_token_id:
            decoded_sentence.append(sampled_token_id)

        # 종료 조건 검사
        if sampled_token_id == stop_token_id or len(decoded_sentence) >= max_songs:
            stop_condition = True

        # 시퀀스와 상태 업데이트
        target_seq[0, 0] = sampled_token_id
        state_h, state_c = h, c

    return decoded_sentence

# 3. inference 실행 : 입력된 값을 출력(수열 형태의 토큰 출력)

In [38]:
from keras.preprocessing.sequence import pad_sequences

# 입력 데이터 준비
input_data = "겨울 노래" # 여기가 사용자 input(query)를 받는 부분이 될 것.

input_seq = playlist_title_tokenizer.encode(input_data, 
                                            add_special_tokens= True  # 향후 수정대상.
                                            )
input_seq_padded = pad_sequences([input_seq], maxlen=max_playlist_title_length, padding='post')

# 인퍼런스 실행
decoded_sequence = inference_songs(input_seq_padded)

# 결과 출력, 얘는 디버깅 출력으로 활용할 것.
print("입력 시퀀스:", input_seq)
print("디코딩된 시퀀스:", decoded_sequence)

1/1 [==============================] - 0s 23ms/step
입력 시퀀스: [101, 8877, 78123, 99706, 102]
디코딩된 시퀀스: [8855, 8247, 14297, 14297, 6651, 14297, 4885, 14297]


# 4. inference -> 노래이름 변환

In [39]:
# idetifier_ 노래이름을 토크나이저로 변환 후, 다시 노래제목으로 처리하기 위한 딕셔너리

def find_key_by_value(value_to_find, dictionary):
    for key, value in dictionary.items():
        if value == value_to_find:
            return key
    return "Unknown Identifier"  # 값이 사전에 없는 경우

# 예시 사용
# identifier = 'song_name_1202'
# song_title = find_key_by_value(identifier, song_identifiers)
# print("노래 제목:", song_title)

In [40]:
def convert_sequence_to_titles(sequence, identifier_to_song):
    song_titles = []
    for seq in sequence:
        
        # 숫자 인덱스를 식별자 형태로 변환
        identifier = f'song_name_{seq}'
        
        # 식별자를 노래 제목으로 변환
        song_title = find_key_by_value(identifier, identifier_to_song)
        song_titles.append(song_title)
    return song_titles

In [50]:
input_data = "눈 오늘 날" # 여기가 사용자 input(query)를 받는 부분이 될 것.

input_seq = playlist_title_tokenizer.encode(input_data, 
                                            add_special_tokens= True  # 향후 수정대상.
                                            )
input_seq_padded = pad_sequences([input_seq], maxlen=max_playlist_title_length, padding='post')

# 인퍼런스 실행
decoded_sequence = inference_songs(input_seq_padded)

song_titles = convert_sequence_to_titles(decoded_sequence, song_identifiers)

song_titles

1/1 [==============================] - 0s 31ms/step


["A Winter Story (From 'Love Letter' Soundtrack)",
 '19금 Holy Grail (Feat. Justin Timberlake)',
 '夜に駆ける',
 '夜に駆ける',
 'Someday My Prince Will Come',
 '19금 Holy Grail (Feat. Justin Timberlake)',
 "Beethoven : Piano Sonata No.14 In C Sharp Minor Op.27-2 'Moonlight' - I. Adagio Sostenuto (베토벤 : 피아노 소나타 14번 올림 다단조 작품번호 27-2 '월광' - 1악장)",
 '돌아가자']

In [42]:
type(song_titles)

list

In [43]:
# 순서를 유지하면서 중복 제거
unique_song_titles_ordered = []
for title in song_titles:
    if title not in unique_song_titles_ordered:
        unique_song_titles_ordered.append(title)

# 결과 출력
unique_song_titles_ordered

["A Winter Story (From 'Love Letter' Soundtrack)",
 '19금 Holy Grail (Feat. Justin Timberlake)',
 '夜に駆ける',
 'Someday My Prince Will Come',
 "Beethoven : Piano Sonata No.14 In C Sharp Minor Op.27-2 'Moonlight' - I. Adagio Sostenuto (베토벤 : 피아노 소나타 14번 올림 다단조 작품번호 27-2 '월광' - 1악장)",
 '돌아가자']